In [2]:
import matplotlib.pyplot as plt
from workflow.scripts.plotting_tools import create_facet_plot
import xarray as xr
from workflow.scripts.utils import calc_relative_change, calc_abs_change
from workflow.scripts.plotting_tools import global_map
import matplotlib as mpl
import numpy as np

In [3]:
nmodels = len(snakemake.input.path_ctrl)
ctrl_path = snakemake.input.path_ctrl
ctrl_path=sorted(ctrl_path)
exp_path = snakemake.input.path_exp
exp_path=sorted(exp_path)
mins=[]
maxs=[]
diffs = {}
for ctrl, exp in zip(ctrl_path, exp_path):
    ds_exp = xr.open_dataset(exp).load()
    ds_ctrl = xr.open_dataset(ctrl).load()
    with xr.set_options(keep_attrs=True):
        if snakemake.wildcards.kind == 'rel':
            diff = calc_relative_change(ds_ctrl, ds_exp)
        elif snakemake.wildcards.kind == 'abs':
            diff = calc_abs_change(ds_ctrl, ds_exp)
            diff[diff.variable_id] = diff[diff.variable_id]
        mins.append(diff[diff.variable_id].min())
        maxs.append(diff[diff.variable_id].max())
        diffs[ds_exp.source_id] = diff
    

In [4]:
minv = np.floor(np.array(mins).mean())

maxv = np.ceil(np.array(maxs).mean())

In [10]:
fig,ax = create_facet_plot(nmodels, figsize=(16,10))
cmap = mpl.cm.bwr


# norm = mpl.colors.BoundaryNorm(np.linspace(-100,100,17),17)
if snakemake.wildcards.kind == 'rel':
    norm = mpl.colors.TwoSlopeNorm(vmin=-30., vcenter=0., vmax=50.)
    label = '$\Delta$ [CLT] %'
else:
    norm = mpl.colors.TwoSlopeNorm(vmin=minv, vcenter=0., vmax=maxv)
    label ='$\Delta$ [CLT]'
for ds, axi in zip(diffs, ax):
    temp_ds = diffs[ds]
    da = temp_ds[temp_ds.variable_id] 
    pcm=temp_ds[temp_ds.variable_id].plot(ax=ax[axi], cmap=cmap, norm=norm,  add_colorbar=False)
    ax[axi].set_title(ds)
    global_map(ax[axi])

cax = fig.add_axes([0.94,0.2,0.02,0.62])
fig.colorbar(pcm, cax=cax, extend='both',
             label=label)
plt.savefig(snakemake.output.outpath, bbox_inches='tight', dpi=144, facecolor='w')
